In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import pytorch_utils
DATA_DIR = 'data/'

[PyTorch-Utils]: Loading anchor candidate data...
[PyTorch-Utils]: Loading item_id to description embedding data...
[PyTorch-Utils]: Loading wikipedia title embedings...
[PyTorch-Utils]: Loading wikipedia items...


In [2]:
# is cuda available?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = pytorch_utils.EntityDataset(device='cuda')
test_dataset = pytorch_utils.EntityDataset(train=False, device='cuda')

[PyTorch-Utils]: Loading all-MiniLM-L6-v2 model & Generating sentence embeddings of train set...


100%|██████████| 7375/7375 [00:04<00:00, 1511.29it/s]


Batches:   0%|          | 0/231 [00:00<?, ?it/s]

[PyTorch-Utils]: Now generating Doc embeddings...


100%|██████████| 946/946 [00:07<00:00, 124.79it/s]


...

[PyTorch-Utils]: Now generating Doc embeddings...


100%|██████████| 447/447 [00:02<00:00, 218.60it/s]


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

[PyTorch-Utils]: Now generating entity embeddings...


Batches:   0%|          | 0/287 [00:00<?, ?it/s]

### my tests

In [26]:
from sentence_transformers import SentenceTransformer

temp_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

In [ ]:
train = pd.read_csv(DATA_DIR + 'train.csv')

In [ ]:
# create a new column in dataframe called doc_id which is the index of the document
# we will do it buy cumsum() the token column. If the value in the token column contains the string -DOCSTART-,
# Then we know we are in a new document
train['doc_id'] = (train['token'].str.contains('-DOCSTART-', case=True, na=False)).cumsum()

In [ ]:
train

,id,token,entity_tag,full_mention,wiki_url,doc_id
0,1,-DOCSTART- (1 EU),NaN,NaN,NaN,1
1,2,EU,B,EU,--NME--,1
2,3,rejects,NaN,NaN,NaN,1
3,4,German,B,German,http://en.wikipedia.org/wiki/Germany,1
4,5,call,NaN,NaN,NaN,1
...,...,...,...,...,...,...
218500,218501,Swansea,B,Swansea,http://en.wikipedia.org/wiki/Swansea_City_A.F.C.,946
218501,218502,1,NaN,NaN,NaN,946
218502,218503,Lincoln,B,Lincoln,http://en.wikipedia.org/wiki/Lincoln_City_F.C.,946
218503,218504,2,NaN,NaN,NaN,946


In [22]:
not_nan = train['wiki_url'].notna()
not_nme = train['wiki_url'] != '--NME--'

# Now for each row that is not_nan & not_nme, we check the token column. If the token column has any previous token with the same value and same doc_id, then we set the full_mention column to be the same as the previous row. Otherwise, we set the full_mention column to be the same as the token column
train['full_mention'] = train[not_nan & not_nme].apply(lambda x: x['token'] if train.loc[(train['token'] == x['token']) & (train['doc_id'] == x['doc_id']), 'token'].count() > 1 else x['full_mention'], axis=1)

KeyboardInterrupt: 

In [8]:
corpus_embeddings = temp_model.encode(wiki_items['wikipedia_title'].to_list(), show_progress_bar=True)

In [9]:
from sentence_transformers import util

query = "Costas Skandalidis"

query_embedding = temp_model.encode(query, convert_to_tensor=True)

corpus_embeddings = torch.tensor(corpus_embeddings).to('cuda')
query_embedding = torch.tensor(query_embedding).to('cuda')
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3, score_function=util.dot_score)

C:\Users\aybar\AppData\Local\Temp\ipykernel_123484\2787295153.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  corpus_embeddings = torch.tensor(corpus_embeddings).to('cuda')
C:\Users\aybar\AppData\Local\Temp\ipykernel_123484\2787295153.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_embedding = torch.tensor(query_embedding).to('cuda')


### actual training

In [4]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_train(x, device=device))
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_test(x, device=device))

In [5]:
model = pytorch_utils.EntityClassifier(device=device)

In [6]:
# loss and optimizer
from torch import optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop
from tqdm import tqdm


epochs = 20
pbar = tqdm(range(epochs))
model.train()
for epoch in pbar:
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model((document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    pbar.set_description(f'Epoch {epoch} loss: {running_loss / len(dataloader)}')

  0%|          | 0/20 [00:00<?, ?it/s]c:\Users\aybar\Documents\CS423-Project-3\pytorch_utils.py:225: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  document_embed = torch.tensor(document_embed, dtype=torch.float32, device=device)
Epoch 19 loss: 0.10277010971139726: 100%|██████████| 20/20 [06:22<00:00, 19.14s/it]


In [7]:
# let's do predictions on the test set
model.eval()
with torch.no_grad():
    predictions = []
    prediction_indexes = []
    for i, data in tqdm(enumerate(test_dataloader)):
        document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings = data
        outputs = model((document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings))
        # which index has the highest value?
        _, predicted = torch.max(outputs, 1)
        prediction_indexes.extend(predicted.tolist())
        # predicted is shape (batch_size)
        # we want to grab the best candidate for batch item i
        # i.e the candidate_ids is of shape (batch_size, 5)
        # we want to grab the best candidate for batch item i
        # i.e get it to shape (batch_size)
        best_candidates = candidate_ids[torch.arange(candidate_ids.size(0)), predicted]

        # Append the best candidates to the predictions list
        predictions.extend(best_candidates.tolist())
        

144it [00:08, 16.83it/s]


In [8]:
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=False, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_train(x, device=device))

In [9]:
# let's do predictions on the test set
model.eval()
with torch.no_grad():
    predictions_train = []
    prediction_train_indexes = []
    for i, data in tqdm(enumerate(train_dataloader)):
        document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings, labels = data
        outputs = model((document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings))
        # which index has the highest value?
        _, predicted = torch.max(outputs, 1)
        prediction_indexes.extend(predicted.tolist())
        # predicted is shape (batch_size)
        # we want to grab the best candidate for batch item i
        # i.e the candidate_ids is of shape (batch_size, 5)
        # we want to grab the best candidate for batch item i
        # i.e get it to shape (batch_size)
        best_candidates = candidate_ids[torch.arange(candidate_ids.size(0)), predicted]

        # Append the best candidates to the predictions list
        predictions_train.extend(best_candidates.tolist())


286it [00:16, 17.21it/s]


In [10]:
# predictions_from_statistics = []

# for i, row in tqdm(test_dataset.entity_df.iterrows()):
#     full_mention = row['full_mention'].strip().lower()
#     if full_mention in pytorch_utils.anchor_to_candidate:
#         best_candidate = pytorch_utils.anchor_to_candidate[full_mention][0]
#     else:
#         best_candidate = 0
#     predictions_from_statistics.append(best_candidate)

In [11]:
wiki_items = pd.read_csv(DATA_DIR + 'wiki_lite/wiki_items.csv')
# index wiki_items by id
wiki_items = wiki_items.set_index('item_id')
# Create item_id to wikipedia_title map
item_id_to_title = wiki_items['wikipedia_title'].to_dict()

In [12]:
enwiki_redirects = pd.read_csv(DATA_DIR + 'wiki_lite/enwiki_redirects.tsv', sep='\t', header=None, names=['source', 'target'])
# index enwiki_redirects by source
enwiki_redirects = enwiki_redirects.set_index('source')
# create source to target map
source_to_target = enwiki_redirects['target'].to_dict()

In [13]:
wiki_urls = []
# Now we will map these into wikipedia_urls
for i in tqdm(range(len(predictions))):
    if predictions[i] == 0:
        # if the prediction is 0, we will append a blank url
        wiki_urls.append('NOT_FOUND')
        continue
    wikipedia_title = item_id_to_title[predictions[i]]
    # does this wikipedia title exist in the redirects?
    if wikipedia_title in source_to_target:
        # if it does, we will replace it with the redirect
        new_title = source_to_target[wikipedia_title]
    # Now replace the spaces with underscores
    wikipedia_title = wikipedia_title.replace(' ', '_')
    # And add the wikipedia url
    wiki_urls.append(f'http://en.wikipedia.org/wiki/{wikipedia_title}')

100%|██████████| 9166/9166 [00:00<00:00, 1406695.59it/s]


In [14]:
train_wiki_urls = []
# Now we will map these into wikipedia_urls
for i in tqdm(range(len(predictions_train))):
    if predictions_train[i] == 0:
        # if the prediction is 0, we will append a blank url
        train_wiki_urls.append('NOT_FOUND')
        continue
    wikipedia_title = item_id_to_title[predictions_train[i]]
    # does this wikipedia title exist in the redirects?
    if wikipedia_title in source_to_target:
        # if it does, we will replace it with the redirect
        new_title = source_to_target[wikipedia_title]
    # Now replace the spaces with underscores
    wikipedia_title = wikipedia_title.replace(' ', '_')
    # And add the wikipedia url
    train_wiki_urls.append(f'http://en.wikipedia.org/wiki/{wikipedia_title}')

100%|██████████| 18288/18288 [00:00<00:00, 1685685.47it/s]


In [15]:
test = pd.read_csv(DATA_DIR + 'test.csv')
train = pd.read_csv(DATA_DIR + 'train.csv')

In [16]:
not_nan = test['wiki_url'].notna()
not_nme = test['wiki_url'] != '--NME--'
train_not_nan = train['wiki_url'].notna()
train_not_nme = train['wiki_url'] != '--NME--'
test.loc[(not_nan & not_nme) & (test.id == 65002)]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,?


In [17]:
test.loc[not_nan & not_nme, 'wiki_url'] = wiki_urls
train.loc[train_not_nan & train_not_nme, 'wiki_url'] = train_wiki_urls

In [18]:
# replace NaN or --NME-- with NOT_FOUND
test['wiki_url'] = test['wiki_url'].fillna('NOT_FOUND')
test['wiki_url'] = test['wiki_url'].replace('--NME--', 'NOT_FOUND')
train['wiki_url'] = train['wiki_url'].fillna('NOT_FOUND')
train['wiki_url'] = train['wiki_url'].replace('--NME--', 'NOT_FOUND')

In [19]:
test[test.id == 65002]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,http://en.wikipedia.org/wiki/Dejan_Koturović


In [20]:
' '.join(test.token.fillna('', inplace=False).to_list()[:250])

"-DOCSTART- (947testa CRICKET) CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .  LONDON 1996-08-30  West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .  Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .  After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .  Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .  Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshi

In [21]:
# # One problem, Simmons does not retrieve the cricket player, but querying for Phil Simmons does.
# # TODO: Fix this, one idea is to group by per doc id, check if this token has a better previous full mention
# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['simmons'])])

# print('____')

# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['phil simmons'])])

In [22]:
# now create a .csv file from id, wiki_url
test[['id', 'wiki_url']].to_csv('submission_with_doc.csv', index=False)
train[['id', 'wiki_url']].to_csv('train_with_doc.csv', index=False)